## Columns for each table

### 1. Patients:
1. BIRTHDATE
2. DEATHDATE
3. MARITAL
4. RACE
5. ETHNICITY
6. GENDER
7. ZIP
8. ID

### 2. Allergies
1. START
2. STOP
3. PATIENT
4. CODE
5. DESCRIPTION

### 3. Conditions
1. START
2. STOP
3. PATIENT
4. CODE
5. DESCRIPTION

### 4. Encounters
1. ID
2. START
3. STOP
4. PATIENT
5. ENCOUNTERCLASS
6. CODE
7. DESCRIPTION

### 5. Immunizations
1. DATE
2. PATIENT
3. CODE
4. DESCRIPTION

### 6. Medications
1. START
2. STOP
3. PATIENT
4. CODE
5. DESCRIPTION

### 7. Observations
1. DATE
2. PATIENT
3. CODE
4. DESCRIPTION
5. VALUE
6. UNITS
7. TYPE

### 8. Procedures
1. DATE
2. PATIENT
3. CODE
4. DESCRIPTION
5. REASONCODE
6. REASONDESCRIPTION


In [0]:
CREATE OR REPLACE TABLE allergies_codes AS
SELECT DISTINCT CODE, DESCRIPTION FROM allergies

In [0]:
CREATE OR REPLACE TABLE conditions_codes AS
SELECT DISTINCT CODE, DESCRIPTION from conditions

In [0]:
CREATE OR REPLACE TABLE encounters_codes AS
SELECT DISTINCT CODE, DESCRIPTION FROM encounters

In [0]:
CREATE OR REPLACE TABLE immunizations_codes AS
SELECT DISTINCT CODE, DESCRIPTION FROM immunizations

In [0]:
CREATE OR REPLACE TABLE medications_codes AS
SELECT CODE, DESCRIPTION FROM medications

In [0]:
CREATE OR REPLACE TABLE observations_codes AS
SELECT
  CODE,
  DESCRIPTION,
  ROW_NUMBER() OVER (ORDER BY CODE, DESCRIPTION) AS ID
FROM observations
GROUP BY CODE, DESCRIPTION;

In [0]:
SELECT * FROM observations_codes;

In [0]:
-- distinct procedure reason codes with description
CREATE OR REPLACE TABLE procedures_reason_codes AS
SELECT DISTINCT REASONCODE, REASONDESCRIPTION FROM procedures;

-- distinct procedure codes with description
CREATE OR REPLACE TABLE procedures_codes AS
SELECT DISTINCT CODE, DESCRIPTION FROM procedures;

In [0]:
SELECT DISTINCT UNITS FROM observations

In [0]:
DESCRIBE TABLE allergies;
DESCRIBE TABLE conditions;
DESCRIBE TABLE immunizations;
DESCRIBE TABLE medications;
DESCRIBE TABLE observations;
DESCRIBE TABLE procedures;

In [0]:
CREATE OR REPLACE TABLE clinical_data AS
SELECT START, STOP, PATIENT, CODE, 'ALLERGIES' AS CLINICALDOMAIN FROM allergies
UNION ALL 
SELECT START, STOP, PATIENT, CODE, 'CONDITIONS' AS CLINICALDOMAIN FROM conditions
UNION ALL
SELECT DATE AS START, NULL AS STOP, PATIENT, CODE, 'IMMUNIZATIONS' AS CLINICALDOMAIN FROM immunizations
UNION ALL
SELECT START, STOP, PATIENT, CODE, 'MEDICATIONS' AS CLINICALDOMAIN FROM medications
UNION ALL
SELECT DATE AS START, NULL AS STOP, PATIENT, oc.ID AS CODE, 'OBSERVATIONS' AS CLINICALDOMAIN
FROM observations o
JOIN observations_codes oc ON o.CODE = oc.CODE
UNION ALL
SELECT DATE AS START, NULL AS STOP, PATIENT, CODE, 'PROCEDURES' AS CLINICALDOMAIN FROM procedures;

In [0]:
SELECT COUNT(*) FROM clinical_data;

In [0]:
%python

catalog = 'workspace'
schema = 'default'
volume_name = 'clinical_data'
spark.sql(f"CREATE VOLUME IF NOT EXISTS {catalog}.{schema}.{volume_name}")

In [0]:
%python

sql_query = 'SELECT * FROM clinical_data'
df = spark.sql(sql_query)
df.display()
output_path = "dbfs:/Volumes/workspace/default/clinical_data/clinical_data.csv"
df.coalesce(1).write.mode("overwrite").option("header", "true").csv(output_path)

Need to find the latest open careplan for each patient